# Candidate Data Transformation
This notebook performs the necessary transformations on the candidate dataset, previously loaded into the `raw_candidates` table in PostgreSQL, to create the final `applicant` table. The transformations include creating a boolean `hired` column based on score criteria, cleaning the data, and preparing it for analysis or visualization in tools like PowerBI.

## Environment Setup
This section imports the required libraries and sets up the connection to the PostgreSQL database, reusing the configuration from notebook 1.

In [18]:
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
engine = create_engine(connection_string)

# Verify the connection
print("Connection to PostgreSQL established successfully.")

Connection to PostgreSQL established successfully.


## Data Extraction
We load the data from the `raw_candidates` table to perform the necessary transformations.

In [19]:
# Load data from raw_candidates
query = "SELECT * FROM raw_candidates;"
df = pd.read_sql(query, engine)

# Rename columns for consistency (same style as in the EDA)
df.rename(columns={
    'First Name': 'first_name',
    'Last Name': 'last_name',
    'Email': 'email',
    'Application Date': 'application_date',
    'Country': 'country',
    'YOE': 'years_of_experience',
    'Seniority': 'seniority',
    'Technology': 'technology',
    'Code Challenge Score': 'code_challenge_score',
    'Technical Interview Score': 'technical_interview_score'
}, inplace=True)

df['application_date'] = pd.to_datetime(df['application_date'], format='mixed')

print("Data extracted successfully. Rows:", len(df))
print("\nUnique technology categories before grouping:", df['technology'].unique())
df.head()

Data extracted successfully. Rows: 50000

Unique technology categories before grouping: ['Data Engineer' 'Client Success' 'QA Manual'
 'Social Media Community Management' 'Adobe Experience Manager' 'Sales'
 'Mulesoft' 'DevOps' 'Development - CMS Backend' 'Salesforce'
 'System Administration' 'Security' 'Game Development'
 'Development - CMS Frontend' 'Security Compliance'
 'Development - Backend' 'Design'
 'Business Analytics / Project Management' 'Development - Frontend'
 'Development - FullStack' 'Business Intelligence'
 'Database Administration' 'QA Automation' 'Technical Writing']


,first_name,last_name,email,application_date,country,years_of_experience,seniority,technology,code_challenge_score,technical_interview_score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7


## Data Transformations
In this section, we transform the data to create the final `applicant` table. This includes creating a boolean `hired` column based on the criteria identified in the EDA (scores >= 7 in both `code_challenge_score` and `technical_interview_score`), removing duplicate emails (if applicable), normalizing data, and grouping technologies into broader categories to facilitate analysis.

In [20]:
# Create the 'hired' column based on EDA criteria
df['hired'] = ((df['code_challenge_score'] >= 7) & (df['technical_interview_score'] >= 7)).astype(bool)

# Remove duplicate emails, keeping the most recent record (based on application_date)
df = df.sort_values('application_date', ascending=False).drop_duplicates(subset='email', keep='first')

# Normalize 'technology' and 'country' columns
df['technology'] = df['technology'].str.strip().str.title()
df['country'] = df['country'].str.strip().str.title()

# Dictionary to map technologies into broader categories
technology_mapping = {
    'Data Engineer': 'Data & Analytics',
    'Client Success': 'Customer Support',
    'Qa Manual': 'Quality Assurance (QA)',
    'Social Media Community Management': 'Marketing & Community',
    'Adobe Experience Manager': 'Marketing & Community',
    'Sales': 'Sales',
    'Mulesoft': 'Integration & Middleware',
    'Devops': 'DevOps & Infrastructure',
    'Development - Cms Backend': 'Development - Backend',
    'Salesforce': 'Customer Relationship Mgmt',
    'System Administration': 'DevOps & Infrastructure',
    'Security': 'Security',
    'Game Development': 'Development - Other',
    'Development - Cms Frontend': 'Development - Frontend',
    'Security Compliance': 'Security',
    'Development - Backend': 'Development - Backend',
    'Design': 'Design',
    'Business Analytics / Project Management': 'Business & Project Management',
    'Development - Frontend': 'Development - Frontend',
    'Development - Fullstack': 'Development - FullStack',
    'Business Intelligence': 'Data & Analytics',
    'Database Administration': 'Data & Analytics',
    'Qa Automation': 'Quality Assurance (QA)',
    'Technical Writing': 'Documentation & Writing'
}

# Apply the mapping to the 'technology' column
df['technology'] = df['technology'].map(technology_mapping)

# Verify the results
print("Number of candidates after removing duplicate emails:", len(df))
print("Percentage of hired candidates:", (df['hired'].mean() * 100).round(2), "%")
print("\nUnique technology categories after grouping:", df['technology'].unique())
df.head()

Number of candidates after removing duplicate emails: 49833
Percentage of hired candidates: 13.41 %

Unique technology categories after grouping: ['Customer Support' 'Marketing & Community' 'Quality Assurance (QA)'
 'Data & Analytics' 'Documentation & Writing' 'Design'
 'Development - Other' 'Sales' 'Business & Project Management' 'Security'
 'DevOps & Infrastructure' 'Customer Relationship Mgmt'
 'Development - Frontend' 'Development - FullStack'
 'Development - Backend' 'Integration & Middleware']


,first_name,last_name,email,application_date,country,years_of_experience,seniority,technology,code_challenge_score,technical_interview_score,hired
23098,Philip,Hessel,kallie.hilpert48@gmail.com,2022-07-04,Uzbekistan,28,Intern,Customer Support,4,5,False
13850,Trystan,Crona,aditya.walsh@gmail.com,2022-07-04,Zambia,23,Intern,Marketing & Community,2,4,False
13653,Clarissa,Rogahn,winston14@hotmail.com,2022-07-04,Brunei Darussalam,25,Architect,Quality Assurance (QA),7,0,False
45936,Maritza,Beer,theodora_grimes28@gmail.com,2022-07-04,Algeria,10,Architect,Data & Analytics,6,8,False
38122,Eulalia,Schaden,chadd_wunsch@gmail.com,2022-07-04,Niger,0,Lead,Quality Assurance (QA),8,9,True


## Loading the Final Table
We create the `applicant` table in PostgreSQL and load the transformed data, which includes all original columns plus the new `hired` column.

In [21]:
# Validate the 'hired' column for null values
if df['hired'].isnull().sum() > 0:
    print("Warning: There are null values in the 'hired' column.")

try:
    df.to_sql('applicant', engine, if_exists='replace', index=False)
    print("Transformed data successfully loaded into the applicant table.")
except Exception as e:
    print(f"Error loading data into PostgreSQL: {e}")

Transformed data successfully loaded into the applicant table.


## Verification of the Final Table
We confirm that the transformed data was correctly loaded into the `applicant` table in PostgreSQL.

In [22]:
# Verify the data in the applicant table
query = "SELECT * FROM applicant LIMIT 5;"
final_df = pd.read_sql(query, engine)
print("First 5 records of the applicant table:")
final_df

First 5 records of the applicant table:


,first_name,last_name,email,application_date,country,years_of_experience,seniority,technology,code_challenge_score,technical_interview_score,hired
0,Philip,Hessel,kallie.hilpert48@gmail.com,2022-07-04,Uzbekistan,28,Intern,Customer Support,4,5,False
1,Trystan,Crona,aditya.walsh@gmail.com,2022-07-04,Zambia,23,Intern,Marketing & Community,2,4,False
2,Clarissa,Rogahn,winston14@hotmail.com,2022-07-04,Brunei Darussalam,25,Architect,Quality Assurance (QA),7,0,False
3,Maritza,Beer,theodora_grimes28@gmail.com,2022-07-04,Algeria,10,Architect,Data & Analytics,6,8,False
4,Eulalia,Schaden,chadd_wunsch@gmail.com,2022-07-04,Niger,0,Lead,Quality Assurance (QA),8,9,True


## Conclusions
This notebook transformed the data from the `raw_candidates` table to create the final `applicant` table, adding the `hired` column based on the established criteria (scores >= 7 in both challenges). Duplicate emails were removed, keeping the most recent record, and certain columns were normalized for consistency. The data is now ready for analysis or visualization in external tools.

### Summary of Transformations
- **Rows Processed**: 49,833 (after removing duplicates).
- **Candidates Hired**: 13.41%.
- **Technology Categories Grouped**: 16 unique categories.

Next steps include generating visualizations or dashboards based on this final table.